# TensorFlow - Save and Load

- #save, #load, #saver, #restore

references:
https://github.com/golbin/TensorFlow-Tutorials

트레이닝 과정중 가장 중요한 일은 텐서플로 환경의 신경망을 저장하는 일이다. 반복적이고 복잡한 계산들을 오랜시간 반복하면서 원하는 단계로 되돌리거나 현 상태를 유지하고싶을때, 혹은 트레이닝된 결과물을 전송하고싶을때 사용된다.

세션을 저장하고 불러오는 과정은 다음과 같다.

``` py
# 세션을 저장할 폴더 위치
path = './model'

# 스텝 수를 저장할 변수를 추가로 정의합니다
global_step = tf.Variable(0, trainable=False, name='global_step')

# 텐서플로 세션 시작
sess = tf.Session()

# 세이버에게 어떤 변수들을 저장하거나 불러올것인지 설정합니다. (tf.global_variables 는 모든 변수를 지칭합니다.)
saver = tf.train.Saver(tf.global_variables())

checkpoint = tf.train.get_checkpoint_state(path) # get_checkpoint_state 는 path 위치에 세션이 저장되어있는지를 판단합니다.
if checkpoint and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, checkpoint.model_checkpoint_path) # 세션이 존재할 경우, restore 함수를 통해 변수들을 초기화합니다.
else:
    sess.run(tf.global_variables_initializer()) # 세션이 존재하지 않는 경우 변수를 초기화합니다.
    
# 세션 진행 (최적화)
for step in range(n_steps):
    sess.run(train_operation,
                feed_dict={X:X_train,Y:y_train})

# 변수를 저장합니다.
saver.save(sess, path + '/network.ckpt', global_step=global_step)
```

주의할 점은 세션을 불러올때 저장되어있는 모델과 현재 정의한 모델과 일치해야한다.

### global_step

global_step 이란 변수는 비록 네트워크에 일부는 아니지만 최적화를 몇번이나 실행했는지를 추적하는 변수를 지정하는것은 좋은 습관이다. 네트워크를 저장할때 몇번이나 최적화가 실행된 네트워크인지를 파일명 뒤에 명시하고, 또한 다른사람이 저장한 네트워크를 사용하거나 추가로 최적화를 할때 쉽게 기록할 수 있다. 이 변수를 세이버에 같이 저장하는것은 어느 순간에 최적화를 정지했는지를 표시하는것과 같다.

### 인터럽트 예외처리

세이버를 사용하는 재밌는 방법중 하나는 Interrupt Exception 안에 세이브를 설정하는것이다. 이렇게 하면 최적화 도중 트레이닝을 중단할때 마지막으로 가지고 있는 네트워크를 저장하게 된다.

``` py
try:
    # 세션 진행 (최적화)
    for step in range(n_steps):
        sess.run(train_operation,
                    feed_dict={X:X_train,Y:y_train})
    # 변수를 저장합니다.
    saver.save(sess, path + '/network.ckpt', global_step=global_step)
    print('최적화 완료. ', sess.run(global_step), '번 최적화 후 저장')
except KeyboardInterrupt:
    # 변수를 저장합니다.
    saver.save(sess, path + '/network.ckpt', global_step=global_step)
    print('최적화 도중 종료.', sess.run(global_step), '번 최적화 후 저장')
```